In [34]:
import os
from pathlib import Path
import os
import pandas as pd

#### The datasets were downloaded from these links (2012 - 2022):
##### Population: https://data.census.gov/table/ACSDP5Y2023.DP05?q=population&g=010XX00US,$0500000 <br> Income: https://data.census.gov/table/ACSST5Y2023.S1903?q=income&g=010XX00US,$0500000 <br> Education: https://data.census.gov/table/ACSST1Y2012.S1501?q=education&g=010XX00US,$0500000 <br> Employment: https://data.census.gov/table/ACSST5Y2023.S2301?q=Employment&g=010XX00US,$0500000 <br> Permits: https://hudgis-hud.opendata.arcgis.com/datasets/HUD%3A%3Aresidential-construction-permits-by-county/about

##### Each of the above categories will have its own folder for the csvs (./population, ./income, etc.) <br> We then have another empty folder ./final_joins that will store the combined featuers for each year (will have just 10 csvs) 

In [ ]:
# Renaming the csv names to make them years after they're downloaded:
# Essentially, rename them to 2021.csv, 2013.csv, etc.
folder = Path("./final_joins")  
for file in folder.glob("*-Data.csv"):
    if file.is_file():
        original_name = file.name
        year = original_name[7:11]  
        new_name = year + ".csv"
        new_path = file.with_name(new_name)
        print(f"Renaming {original_name} -> {new_name}")
        file.rename(new_path)

#### Essentially joined each of the categories above together based on GEO_ID. For each year, the methods below then accumulated the features into 1 csv for that year.

In [ ]:
# Joins the population and income feature csvs

def join_population_income(population_folder, income_folder, output_folder, start_year=2012, end_year=2022):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for year in range(start_year, end_year + 1):
        # Skip the year 2020
        if year == 2020:
            print("Skipping year 2020")
            continue
        
        population_file = f"{year}_cleaned.csv"
        income_file = f"{year}.csv"
        population_file_path = os.path.join(population_folder, population_file)
        income_file_path = os.path.join(income_folder, income_file)
        
        if os.path.isfile(population_file_path) and os.path.isfile(income_file_path):
            try:
                # Read the population and income CSVs
                population_df = pd.read_csv(population_file_path)
                income_df = pd.read_csv(income_file_path)

                # Perform an inner join only on the GEO_ID column
                merged_df = pd.merge(population_df, income_df, on="GEO_ID", how="inner")
                output_file_path = os.path.join(output_folder, f"{year}.csv")

                # Save the merged DataFrame to a new CSV
                merged_df.to_csv(output_file_path, index=False)
                print(f"Successfully merged and saved: {output_file_path}")
            except Exception as e:
                print(f"Error processing year {year}: {e}")
        else:
            print(f"Missing files for year {year}: Population file: {population_file_path}, Income file: {income_file_path}")

population_folder = './population'  
income_folder = './income'        
output_folder = './final_joins'        
join_population_income(population_folder, income_folder, output_folder)

In [ ]:
# Joins the employment feature csvs

def merge_employment_data(final_joins_path, employment_path):
    for year in range(2012, 2023):
        if year == 2020:
            continue  # Skip 2020
        filename = f"{year}.csv"
        final_file = os.path.join(final_joins_path, filename)
        employment_file = os.path.join(employment_path, filename)
        if not os.path.exists(final_file):
            print(f"Missing in final_joins: {filename}")
            continue
        if not os.path.exists(employment_file):
            print(f"Missing in employment: {filename}")
            continue

        try:
            # Load both datasets
            final_df = pd.read_csv(final_file)
            emp_df = pd.read_csv(employment_file)
            employment_columns = emp_df.columns[2:]
            new_columns = list(employment_columns)
            emp_dict = emp_df.set_index(emp_df.columns[0])[new_columns].to_dict(orient='index')

            # Create a DataFrame to hold new columns (same index as final_df)
            new_data = pd.DataFrame(columns=new_columns, index=final_df.index)

            # Fill in the new columns based on GEO_ID matches
            for i, geo_id in enumerate(final_df.iloc[:, 0]):  
                if geo_id in emp_dict:
                    for col in new_columns:
                        new_data.at[i, col] = emp_dict[geo_id].get(col, '')
                else:
                    pass

            # Append new employment columns to final_df
            final_df = pd.concat([final_df, new_data], axis=1)

            # Save updated final_df
            final_df.to_csv(final_file, index=False)
            print(f"Merged employment data into {filename}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")
merge_employment_data('./final_joins', './employment')

✅ Processed: 2021.csv
✅ Processed: 2023.csv
✅ Processed: 2022.csv
✅ Processed: 2019.csv
✅ Processed: 2018.csv
✅ Processed: 2015.csv
✅ Processed: 2014.csv
✅ Processed: 2016.csv
✅ Processed: 2017.csv
✅ Processed: 2013.csv
✅ Processed: 2012.csv
✅ Processed: 2010.csv
✅ Processed: 2011.csv
✅ Finished building mapping. Total unique counties: 873
dict_keys(['Baldwin County, Alabama', 'Calhoun County, Alabama', 'Cullman County, Alabama', 'DeKalb County, Alabama', 'Elmore County, Alabama', 'Etowah County, Alabama', 'Houston County, Alabama', 'Jefferson County, Alabama', 'Lauderdale County, Alabama', 'Lee County, Alabama', 'Limestone County, Alabama', 'Madison County, Alabama', 'Marshall County, Alabama', 'Mobile County, Alabama', 'Montgomery County, Alabama', 'Morgan County, Alabama', 'St. Clair County, Alabama', 'Shelby County, Alabama', 'Talladega County, Alabama', 'Tuscaloosa County, Alabama', 'Anchorage Municipality, Alaska', 'Fairbanks North Star Borough, Alaska', 'Matanuska-Susitna Boroug

In [ ]:
# Joins the education feature csvs

def append_education_data(final_joins_path, education_path):
    for year in range(2012, 2023):
        if year == 2020:
            continue  # Skip 2020
        file_name = f"{year}.csv"
        final_path = os.path.join(final_joins_path, file_name)
        edu_path = os.path.join(education_path, file_name)

        if not os.path.isfile(final_path):
            print(f"final_joins file not found: {file_name}")
            continue
        if not os.path.isfile(edu_path):
            print(f"education file not found: {file_name}")
            continue

        try:
            df_final = pd.read_csv(final_path)
            df_edu = pd.read_csv(edu_path)
            final_id_col = df_final.columns[0]
            edu_id_col = df_edu.columns[0]
            edu_feature_cols = df_edu.columns[2:]

            df_edu_subset = df_edu[[edu_id_col] + list(edu_feature_cols)].copy()
            df_edu_subset.columns = [edu_id_col] + list(edu_feature_cols)  # keep col names unchanged
            df_merged = pd.merge(df_final, df_edu_subset, how='left', left_on=final_id_col, right_on=edu_id_col)
            if final_id_col != edu_id_col and edu_id_col in df_merged.columns:
                df_merged.drop(columns=[edu_id_col], inplace=True)
            df_merged.to_csv(final_path, index=False)
            print(f"Appended education data to: {file_name}")

        except Exception as e:
            print(f"Error processing {file_name}: {e}")
append_education_data('./final_joins', './education')

Filtered and saved: 2021.csv
Filtered and saved: 2023.csv
Filtered and saved: 2022.csv
Filtered and saved: 2019.csv
Filtered and saved: 2018.csv
Filtered and saved: 2015.csv
Filtered and saved: 2014.csv
Filtered and saved: 2016.csv
Filtered and saved: 2017.csv
Filtered and saved: 2013.csv
Filtered and saved: 2012.csv
Filtered and saved: 2010.csv
Filtered and saved: 2011.csv


In [ ]:
# Joins the permits csv 

def add_permits_to_joins(final_joins_folder, supply_file):
    supply_df = pd.read_csv(supply_file)
    supply_df.columns = supply_df.columns.str.strip()
    supply_df.rename(columns={'GEOID': 'GEO_ID'}, inplace=True)

    # Iterate through each CSV file in the final_joins folder (years 2012 to 2022)
    for year in range(2012, 2023):
        if year == 2020:
            continue  # Skip the year 2020
        file_name = f"{year}.csv"
        file_path = os.path.join(final_joins_folder, file_name)

        if os.path.isfile(file_path):
            try:
                df = pd.read_csv(file_path)
                df.columns = df.columns.str.strip()
                if 'GEO_ID' not in df.columns:
                    print(f"'GEO_ID' not found in {file_name}")
                    continue  # Skip this file if 'GEO_ID' is not present
                df['SINGLE_FAMILY_PERMITS'] = None
                for index, row in df.iterrows():
                    geo_id = row['GEO_ID']
                    # Check if the GEO_ID exists in the supply.csv dataset
                    if geo_id in supply_df['GEO_ID'].values:
                        permit_value = supply_df.loc[supply_df['GEO_ID'] == geo_id, f"SINGLE_FAMILY_PERMITS_{year}"].values[0]
                        
                        # Set the permit value in the 'SINGLE_FAMILY_PERMITS' column
                        df.at[index, 'SINGLE_FAMILY_PERMITS'] = permit_value
                # Save the updated DataFrame to the same CSV file
                df.to_csv(file_path, index=False)
                print(f"Updated {file_name}")

            except Exception as e:
                print(f"Error processing {file_name}: {e}")
        else:
            print(f"File not found: {file_name}")

final_joins_folder = './final_joins'
supply_file = './supply.csv'  
add_permits_to_joins(final_joins_folder, supply_file)

❌ Row count mismatch detected:
  2021.csv: 842 rows
  2023.csv: 855 rows
  2022.csv: 849 rows
  2019.csv: 841 rows
  2018.csv: 839 rows
  2015.csv: 831 rows
  2014.csv: 829 rows
  2016.csv: 832 rows
  2017.csv: 838 rows
  2013.csv: 829 rows
  2012.csv: 826 rows
  2010.csv: 819 rows
  2011.csv: 823 rows


In [ ]:
# Removes columns that represent margins of error and percent margins of error since we don't need that
def filter_files(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('_Metadata.csv'):
            file_path = os.path.join(folder_path, filename)
            try:
                df = pd.read_csv(file_path)
                first_col = df.columns[0]
                df_filtered = df[~df[first_col].astype(str).str.endswith(("M", "PE", "PM"))]
                df_filtered.to_csv(file_path, index=False)
                print(f"Filtered and saved: {file_path}")
            except Exception as e:
                print(f"Error processing {file_path}: {e}")
filter_metadata_files("./final_joins")


Filtered and saved: ./population/2017_Metadata.csv
Filtered and saved: ./population/2021_Metadata.csv
Filtered and saved: ./population/2016_Metadata.csv
Filtered and saved: ./population/2011_Metadata.csv
Filtered and saved: ./population/2018_Metadata.csv
Filtered and saved: ./population/2019_Metadata.csv
Filtered and saved: ./population/2013_Metadata.csv
Filtered and saved: ./population/2014_Metadata.csv
Filtered and saved: ./population/2022_Metadata.csv
Filtered and saved: ./population/2015_Metadata.csv
Filtered and saved: ./population/2012_Metadata.csv
Filtered and saved: ./population/2023_Metadata.csv
